# ***Sistema experto para diagnóstico de fallas en un vehículo***

In [ ]:
pip install experta

  Preparing metadata (setup.py) ... done
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3149 sha256=87e49916c4158c78fa155c1206efc0d43d74ff33c25f4134eebe4b9d5de45755
  Stored in directory: /root/.cache/pip/wheels/f6/ff/aa/750fec7bf9618d87b53572def5abf3e098f853cc5ab4147656
Successfully built frozendict
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.6
    Uninstalling frozendict-2.4.6:
      Successfully uninstalled frozendict-2.4.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.65 requires frozendict>=2.3.4, but you have frozendict 1.2 which is incompatible.


In [ ]:
# Para compatibilidad con versiones anteriores, siempre utilizarlo.
import collections.abc
if not hasattr(collections, 'Mapping'):
    collections.Mapping = collections.abc.Mapping

In [ ]:
from experta import *  # Importa la libreria experta para crear el sistema experto

# Definicion de hechos utilizados en el sistema
class Symptom(Fact):  # Representa un sintoma observado en el vehiculo
    pass

class CarState(Fact):  # Estado general del motor u otra parte del vehiculo
    pass

class Diagnosis(Fact):  # Diagnostico generado por el sistema
    pass

class RepairAction(Fact):  # Representa una accion de reparacion necesaria
    pass

class VehicleStatus(Fact):  # Estado general del proceso de reparacion
    pass

# Clase principal que contiene las reglas del sistema experto
class VehicleDiagnosis(KnowledgeEngine):

    # Regla: humo blanco y luz de aceite indican posible dano grave al motor
    @Rule(AND(Symptom(tipo='humo_blanco'), Symptom(tipo='luz_aceite')), salience=40)
    def motor_grave_dano(self):
        resultado = 'Posible_junta_de_cabeza_mala,_motor_en_riesgo'
        print(resultado)
        self.declare(Diagnosis(resultado=resultado))
        self.declare(RepairAction(tipo='reparar_motor'))

    # Regla: ruido metalico indica posible problema en los frenos
    @Rule(Symptom(tipo='ruido_metalico'), salience=50)
    def revisar_frenos(self):
        resultado = 'Revisar_sistema_de_frenos'
        print(resultado)
        self.declare(Diagnosis(resultado=resultado))
        self.declare(RepairAction(tipo='reparar_frenos'))

    # Complete la regla: fuga de liquido + motor caliente = posible perdida de refrigerante
    @Rule(AND(Symptom(tipo='fuga_liquido'), CarState(estado='motor_caliente')), salience=50)
    def perdida_refrigerante(self):
        resultado = 'Perdida_de_refrigerante,_posible_sobrecalentamiento'
        print(resultado)
        self.declare(Diagnosis(resultado=resultado))
        self.declare(RepairAction(tipo='rellenar_refrigerante'))

    # Regla: si no hay sintomas clave, sugerir revision general
    @Rule(NOT(Symptom(tipo='humo_blanco')),NOT(Symptom(tipo='luz_aceite')), NOT(Symptom(tipo='ruido_metalico')), salience=10)
    def revision_general(self):
        resultado = 'Revision_general_recomendada'
        print(resultado)
        self.declare(Diagnosis(resultado=resultado))

    # Regla de reparacion: eliminar sintoma 'ruido_metalico'
    @Rule(RepairAction(tipo='reparar_frenos'), Symptom(tipo='ruido_metalico'), salience=150)
    def ejecutar_reparacion_frenos(self):
        print("Reparando sistema de frenos - Eliminando sintoma 'ruido_metalico'")
        for fact in self.facts:
            if isinstance(fact, Symptom) and self.facts[fact]['tipo'] == 'ruido_metalico':
                self.retract(fact)
                break
        print("Sintoma eliminado, vehiculo reparado\n")
        self.declare(VehicleStatus(estado="verificar_reparacion"))

    # Regla de reparacion: eliminar sintoma 'humo_blanco'
    @Rule(RepairAction(tipo='reparar_motor'), Symptom(tipo='humo_blanco'), salience=150)
    def ejecutar_reparacion_motor_humo(self):
        print("Reparando problema del motor - Eliminando sintoma 'humo_blanco'")
        for fact in self.facts:
            if isinstance(fact, Symptom) and self.facts[fact]['tipo'] == 'humo_blanco':
                self.retract(fact)
                break
        self.declare(VehicleStatus(estado='en_reparacion'))
        print("Sintoma de humo eliminado, motor en reparacion\n")
        self.declare(VehicleStatus(estado="verificar_reparacion"))

    # Regla de reparacion: eliminar sintoma 'luz_aceite'
    @Rule(RepairAction(tipo='reparar_motor'), Symptom(tipo='luz_aceite'), salience=150)
    def ejecutar_reparacion_motor_aceite(self):
        print("Reparando problema del motor - Eliminando sintoma 'luz_aceite'")
        for fact in self.facts:
            if isinstance(fact, Symptom) and self.facts[fact]['tipo'] == 'luz_aceite':
                self.retract(fact)
                break
        print("Sintoma de luz de aceite eliminado\n")
        self.declare(VehicleStatus(estado="verificar_reparacion"))

    # Verifica si ya no quedan sintomas: vehiculo esta listo
    @Rule(VehicleStatus(estado="verificar_reparacion"), salience=5)
    def vehiculo_reparado(self):
        tiene_sintomas = False
        for fact in self.facts:
            if isinstance(fact, Symptom):
                tiene_sintomas = True
                break

        if not tiene_sintomas:
            print("Todos los sintomas han sido resueltos. Vehiculo listo.")

# Ejecucion del sistema experto
if __name__ == "__main__":
    engine = VehicleDiagnosis()
    engine.reset()  # Reinicia el motor

    print("Primera ejecucion - Diagnostico inicial:\n")
    # Complete las declaraciones
    engine.declare(Symptom(tipo ='humo_blanco'))
    engine.declare(Symptom(tipo ='luz_aceite'))
    engine.declare(Symptom(tipo ='ruido_metalico'))

    engine.run()  # Ejecuta las reglas

Primera ejecucion - Diagnostico inicial:

Revisar_sistema_de_frenos
Reparando sistema de frenos - Eliminando sintoma 'ruido_metalico'
Sintoma eliminado, vehiculo reparado

Posible_junta_de_cabeza_mala,_motor_en_riesgo
Reparando problema del motor - Eliminando sintoma 'luz_aceite'
Sintoma de luz de aceite eliminado

Reparando problema del motor - Eliminando sintoma 'humo_blanco'
Sintoma de humo eliminado, motor en reparacion

Todos los sintomas han sido resueltos. Vehiculo listo.


## **Preguntas:**

### **1. ¿Qué resultado se obtiene si se agrega un nuevo síntoma como ruido metalico en la entrada? Explique su respuesta**

##### ***Rta:** El sistema dispara la regla de falla moderada `(Symptom(tipo='ruido_metalico'))`. Se genera el diagnóstico Diagnosis`(resultado='Revisar_sistema_de_frenos')` y se activa una acción de reparación `RepairAction(tipo='reparar_frenos')`.*
<br>

### **2. Utilizando las declaraciones anteriores, ¿Qué sucede si se cambia la saliencia de motor_grave_dano de 100 a 40?**

##### ***Rta:** La regla de frenos (50) se ejecuta antes que la del motor (40), ya que la salience indica prioridad, a mayor número, mayor prioridad. Al bajarla a 40, la falla grave del motor se atiende después que la moderada de frenos.*
<br>

### **3. ¿Qué sucede si eliminamos salience de todas las reglas?**

##### ***Rta:** Todas las reglas quedan con prioridad 0, entonces el motor las ejecuta según orden de activación y la política FIFO (Last-In, First-Out).*
<br>

### **4. ¿Qué ocurre si se activan múltiples reglas dentro del sistema experto, y cada una declara un hecho del mismo tipo Diagnosis, pero con diferentes valores en su atributo resultado, como `Diagnosis(resultado=...)`? ¿Se almacenan todos los diagnósticos generados, se sobrescribe alguno, o el motor de inferencia solo considera uno de ellos?**

##### ***Rta:** Se almacenan todos los diagnósticos generados como hechos distintos.<br> <br> **1**. Experta mantiene una memoria de trabajo (un “bolsillo” de hechos). <br> **2**. Cada `self.declare(Diagnosis(resultado=...))` agrega un hecho nuevo al bolsillo. <br> **3**. No hay “variable única Diagnosis”; son hechos independientes. <br> **4**. Solo se eliminan si tú llamas a `self.retract(...)`.*
<br>

### **5. ¿Cómo cambia el comportamiento del sistema cuando se ejecuta nuevamente después de eliminar algunos síntomas mediante retract?**

##### ***Rta:** El sistema ya no activará las reglas asociadas a los síntomas eliminados, al retractar un hecho, se elimina de la memoria del motor, entonces las reglas que dependían de él dejan de cumplirse. El flujo de diagnósticos y reparaciones será más corto o diferente según qué síntomas se retiren.*
<br>

### **6. ¿Qué ventajas presenta el uso de un hecho intermedio (RepairAction) para gestionar las reparaciones?**

##### ***Rta:** Algunas de las ventajas que presenta son: <br> <br> **1.** Separación de responsabilidades <br> <li> Regla de diagnóstico: decide qué pasa. <br> <li> Regla de reparación: decide qué hacer. <br> <br> **2.** Modularidad y escalabilidad <li> En un futuro si agregamos otra reparación para el mismo diagnóstico, solo añaderiamos una regla que reaccione a `RepairAction(...)`, sin tocar las reglas de diagnóstico.*